In [32]:
# Coding Challenge

import pandas as pd
import numpy as np
import altair as alt
import altair_saver

dat = pd.read_csv('https://github.com/byuidatascience/data4missing/raw/master/data-raw/mtcars_missing/mtcars_missing.csv')

dat.car = dat.car.dropna()
dat.hp = dat.hp.dropna()
empty = np.NaN
# print(dat.query('car != np.NaN'))
dat = dat.sort_values(by = "car")
dat_2 = dat.drop(dat.index[[28,29,30,31]])

dat_2 = dat_2.sort_values(by = "hp")

hp_mean = dat_2.hp.dropna().mean()



dat_2.hp = dat_2.hp.replace(['nan', np.nan], hp_mean)

# mean_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.replace(-999, np.nan).mean()

In [ ]:
url = 'https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url)

flights

# we can use .describe or .value_counts to learn more about the data


flights.describe()

flights.groupby("airport_code").num_of_delays_total.agg("max")

# -999 is one way to code missing data.  Especially social sciences
# What does 1500+ mean?  Greater than 1500?  How do you treat it, because technically some data is missing 
# the value it is about 1500 is not included
# nan is a missing value and n/a
# look for columns that are text but should be numbers.  They may have missing value identifiers like Nan forcing them to be text

# for question 1 calculate likelihood of a disruptive delay, which we would define.  (somewhere between 10-30 minutes I think)

# .pop removed one item from a list for your perusal

flights.assign(
   Year2= lambda x: x.year * 2 
).Year2

In [ ]:
# Power Summary Function


# inputs: dataframe and possibly list some specific columns
# outputs would be graphical summary like GGally GGpairs ## and a numerical summary by column (min, max, average, total, STDev, missing values, data type)


def power_summary(df):

    print("This is the Numerical Summary Portion")

    df.describe()

    df.head()

    df.info()

    df.nunique()

    df.count()  #any column with less rows counted has missing or NA values

    # value_counts this is to use on a column.  We would have to iterate through a list of columns and apply it to each
    # maybe iterate through with .min .max .mean and .median  value_counts kind of like the summary function in R
    # df.groupby("sort_column").agg(['max', 'min', 'median'])
    # look at using lambda functions or .assign to simplify the coding of the power_summary function

power_summary(flight)

In [ ]:
url2 = "https://github.com/byuidatascience/data4missing/raw/master/data-raw/mtcars_missing/mtcars_missing.json" 

cars = pd.read_json(url2)

cars.groupby("gear").agg(['max', 'min', 'median'])

cars.describe()

In [ ]:
# 2/1/2022 Class Period

# We can choose decimals with round function and specify number of decimal places.

# cars.head()

# cars.cyl.value_counts()

# cars.groupby('cyl').agg(hp_ave = ('hp', np.mean)).round(2)

cyl_data = cars.groupby('cyl').agg(hp_ave = ('hp', np.mean)).reset_index()

# :O is ordinal, :Q is quantitative, :T is time

# when we groupby we turn the grouping variable into the index
# to turn the index back into a regular column we need to add .reset_index() at the end

alt.Chart(cyl_data).mark_point().encode(x = 'cyl:Q', y = alt.Y('hp_ave', axis = alt.Axis(format = '%')))

# for question 2 just drop the rows in month that have n/a's.

flights.query('month != "n/a"')


# we can replace with np.nan to make it an official missing data value

flights.replace([-999, "1500+"], np.nan)

# be wary of unofficial missing values forcing the column to be a string.
# you need to remove or replace the missing value and then reformat the column as the proper data type


# .fillna just fills na's with the value above them in the column  (for the ffill method,  fillna has many methods)
# bfill method fills with the value below the n/a


# from collections import counter  (creates a list of counts)




In [ ]:
# Question 1

# Which airport has the worst delays? Discuss how you chose to define “worst”. 
# Your answer should include a summary table that lists (for each airport) the total number of flights, 
# total number of delayed flights, proportion of delayed flights, and average delay time in hours.


import pandas as pd
import numpy as np
import altair as alt
import altair_saver
from collections import Counter

url_f = 'https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url_f)

flights_2 = flights

# Counter(flights.month)

flights.isnull().sum()


flights.groupby('airport_name').minutes_delayed_total.mean()

# flights.groupby("airport_name").filter([]).agg(['sum', 'min', 'median'])
flights_grouped = flights_2.groupby('airport_code')

flights_q1 = flights_grouped.agg(
  Total_Flights = ('num_of_flights_total', sum),
  Total_Delayed_Flights = ('num_of_delays_total', sum),
  Total_Mins_Delay = ('minutes_delayed_total', sum)).assign(
  Perc_of_Flights_Delayed = lambda x: x.Total_Delayed_Flights / x.Total_Flights,
  Mean_Delay_in_Hours = lambda x: ( x.Total_Mins_Delay/ x.Total_Delayed_Flights)/60  
  )

print(flights_q1.to_markdown())


# Question Two

flights_by_month = flights.query('month != "n/a"').groupby('month').agg(
  Total_Flights = ('num_of_flights_total', sum),
  Total_Delayed_Flights = ('num_of_delays_total', sum)).assign(
  Perc_of_Flights_Delayed = lambda x: x.Total_Delayed_Flights / x.Total_Flights
  ).reset_index()

flights_by_month
months = ["January", "Febuary", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
flights_q2_chart = alt.Chart(flights_by_month, title = "% Delayed Flights by Month").mark_bar().encode(x = alt.X('month', sort = months ), y = 'Perc_of_Flights_Delayed')


flights_q2_chart.save('Flights_Q2_Chart.png')


# Question Three

mean_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.replace(-999, np.nan).mean()
flights_q3 = flights
flights_q3.num_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.replace(-999, mean_of_delays_late_aircraft) 

# 100% of delayed flights in the Weather category are due to weather.
# 30% of delayed flights in the Late-Arriving category are due to weather.
# From April to August, 40% of delayed flights in the NAS category are due to weather. The rest of the months, the proportion rises to 65%.


weather = flights_q3.assign(
    severe = lambda x: x.num_of_delays_weather,
    mild_late = (0.3 * flights_q3.num_of_delays_late_aircraft).round(),
    mild_nas = np.where(flights_q3.month == ("April" or "May" or "June" or "July" or "August"), 0.4 * flights_q3.num_of_delays_nas, 0.65 * flights_q3.num_of_delays_nas).round(),
   total_weather = lambda x: x.severe + x.mild_late + x.mild_nas,
).filter(['airport_code','month','severe','mild_late','mild_nas',
   'total_weather', 'num_of_delays_total', 'num_of_flights_total'])


weather_2 = weather.groupby("airport_code").agg(
  Total_Weather_Delays = ('total_weather', sum),
  Total_Flights = ('num_of_flights_total', sum)).assign(total_weather_perc = lambda x: x.Total_Weather_Delays/x.Total_Flights).reset_index()


weather_2_chart = alt.Chart(weather_2, title = "% Delayed Due To Weather").mark_bar().encode(x = 'airport_code', y = alt.Y('total_weather_perc', axis=alt.Axis(format='%')))

weather_2_chart.save('Weather_2_Chart.png')








In [9]:
# Question 5

import pandas as pd
import numpy as np
import altair as alt
import altair_saver
from collections import Counter

url_f = 'https://raw.githubusercontent.com/byuidatascience/data4missing/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url_f)


flights_clean = flights.replace([-999, "1500+",'n/a', 'nan',"NaN","",np.nan, pd.NA], np.NaN)

flights_clean.query('index == 2').head(1).to_json('q_5_json.json')





In [5]:
# Question 3 Template from class


mean_of_delays_late_aircraft = flights.num_of_delays_late_aircraft.mean()

weather = flights.assign(
    severe = #????,
    mild_late = #????,
    mild_nas = np.where(#????),
   total_weather = # add up severe and mild,
).filter(['airport_code','month','severe','mild_late','mild_nas',
   'total_weather', 'num_of_delays_total']))

# Cars data practice for question 3

# check cars data for missing data

cars.isnull().sum()
mean_of_gear = cars.gear.replace(999, np.nan).mean()

cars_2 = cars
cars_2.gear = cars.gear.replace(999, mean_of_gear)


# Make a new column "bob" that is sum of:
# 100% of cyl, +
# 60% of gear, +
# 100% of carb if disp < 200, but 200% of carb if disp >= 200


answer = cars_2.assign(
  part1 = cars_2.cyl,
  part2 = 0.6 * cars_2.gear,
  part3 = np.where(cars_2.disp < 200, cars_2.carb, 2 * cars_2.carb),
  bob = lambda x: x.part1 + x.part2 + x.part3
)

print(answer.head().filter(["car", "cyl", "disp", "carb", "bob", "part1", "part2", "part3"]).to_markdown())

# Question 4 is aggregate the weather delays data by airport

cars_2.assign(
  part1 = cars_2.cyl,
  part2 = 0.6 * cars_2.gear,
  part3 = np.where(cars_2.disp < 200, cars_2.carb, 2 * cars_2.carb),
  bob = lambda x: x.part1 + x.part2 + x.part3
)

|    | car               |   cyl |   disp |   carb |   bob |   part1 |   part2 |   part3 |
|---:|:------------------|------:|-------:|-------:|------:|--------:|--------:|--------:|
|  0 | Mazda RX4         |     6 |    160 |      4 |  12.4 |       6 |     2.4 |       4 |
|  1 | Mazda RX4 Wag     |     6 |    160 |      4 |  12.4 |       6 |     2.4 |       4 |
|  2 | Datsun 710        |     4 |    108 |      1 |   7.1 |       4 |     2.1 |       1 |
|  3 | Hornet 4 Drive    |     6 |    258 |      1 |   9.8 |       6 |     1.8 |       2 |
|  4 | Hornet Sportabout |     8 |    360 |      2 |  13.8 |       8 |     1.8 |       4 |


,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,part1,part2,part3,bob
0,Mazda RX4,21.0,6,160.0,110.0,3.90,2.620,16.46,0.0,1,4.0,4,6,2.4,4,12.4
1,Mazda RX4 Wag,21.0,6,160.0,110.0,3.90,2.875,17.02,NaN,1,4.0,4,6,2.4,4,12.4
2,Datsun 710,22.8,4,108.0,93.0,3.85,2.320,18.61,1.0,1,3.5,1,4,2.1,1,7.1
3,Hornet 4 Drive,21.4,6,258.0,110.0,3.08,NaN,19.44,1.0,0,3.0,1,6,1.8,2,9.8
4,Hornet Sportabout,18.7,8,360.0,175.0,3.15,3.440,17.02,0.0,0,3.0,2,8,1.8,4,13.8
5,Valiant,18.1,6,225.0,105.0,2.76,NaN,20.22,1.0,0,3.0,1,6,1.8,2,9.8
6,,14.3,8,360.0,NaN,3.21,NaN,15.84,0.0,0,3.0,4,8,1.8,8,17.8
7,,24.4,4,146.7,62.0,3.69,3.190,20.00,1.0,0,4.0,2,4,2.4,2,8.4
8,Merc 230,22.8,4,140.8,95.0,3.92,3.150,22.90,1.0,0,4.0,2,4,2.4,2,8.4
9,Merc 280,19.2,6,167.6,123.0,3.92,NaN,18.30,1.0,0,4.0,4,6,2.4,4,12.4
